# Calculate the relative rates of different supernovae as a function of redshift

Supernova rates are calculated by convolving the Delay time distribution with the star formation history. 
We then set the relative rates of SN Ia and Core Collapse to be 25% and 75%, respectively, at z = 0. 


References:
- Delay time distribution (DTD) for SN Ia: Graur et al (2013) https://arxiv.org/abs/1310.3495
- Delay time distribution (DTD) for SN II: Zapartas et al (2017) https://arxiv.org/abs/1701.07032
- Cosmic Star Formation Rate out to z = 8 : Behroozi et al (2013) https://arxiv.org/abs/1207.6105

In [ ]:
%pylab inline
import numpy as np
from scipy.stats import norm

## Calculate the Lookback Time in units of Gyr using LCDM cosmology

In [ ]:
c1 = 2.9979E5 #km / s
pc= 3.085677E16 #m
c2=pc*c1*1e6
H0=70.
Om=0.30
Ol=0.7


def Efunc(x):
    return math.sqrt(Ol+Om*(1+x)**3)


convert=3600*24*365.25*10**9/(pc*10**3)#*10**9 ## Converts H0 to 1/gigayears

def t_L(z):
    t = np.zeros(len(z))
    import scipy.integrate as integrate
    for i in range(len(z)):
        x = z[i]
        test= lambda x:1.0/((1+x)*Efunc(x))
        y, err=integrate.quad(test, 0, x)
        t[i] = y/H0/convert
    return t 

In [ ]:
# Star formation history = cosmic star formation rate.
# From Behroozi et al (2013)
# Note this is a function of redshift
def cosmic_SFR_B13(z):
    z0 = 1.243
    A = -0.997
    B = 0.241
    C = 0.180
    return C/(10.**(A*(z-z0)) + 10.**(B*(z-z0)))*(1./0.7)

# From Madau and Dickinson (2014)
def cosmic_SFR(z):
    return 0.015 * (1.+z)**2.7/(1.0+((1.+z)/2.9)**5.6) # M_star/year/Mpc^3

In [ ]:
# From Graur et al (2013). There are many, many different forms this can take. 
## The extra term at the end is a normalization factor
def DTD_Ia_Power(delta_t, beta = -1.47):
    return ((delta_t < 1.7)*delta_t**(-0.5) + (delta_t > 1.7)*delta_t**(beta))*0.000274005476758


# From Zapartas et al (2017). I suggest using the binary population, but you can you single star evolution instead. See paper for 
# details. I left both in because it's complicated to type and has different units.
def DTD_cc_single(delta_t): # roughly 1/3 for Ib/c
    delta_t = delta_t*1000.
    return ((delta_t >= 3) & (delta_t  < 25))*1e-9*(-2.83 + 8.7*np.log10(delta_t) - 2.07*np.log10(delta_t)*np.log10(delta_t))*(1./delta_t) + \
        ((delta_t >= 25) & (delta_t  < 48))*1e-8*(-4.85 + 6.55*np.log10(delta_t) - 1.92*np.log10(delta_t)*np.log10(delta_t))*(1./delta_t)

def DTD_cc_binary(delta_t): # roughly 1/3 for Ib/c
    delta_t = delta_t*1000.
    return ((delta_t >= 3) & (delta_t  < 25))*1e-9*(-2.65 + 7.51*np.log10(delta_t) - 0.98*np.log10(delta_t)*np.log10(delta_t))*(1./delta_t) + \
        ((delta_t >= 25) & (delta_t  < 48))*1e-8*(-0.89 + 1.73*np.log10(delta_t) - 0.51*np.log10(delta_t)*np.log10(delta_t))*(1./delta_t) + \
        ((delta_t >= 48) & (delta_t  < 200))*1e-8*(3.46 - 2.98*np.log10(delta_t) + 0.65*np.log10(delta_t)*np.log10(delta_t))*(1./delta_t)
        
        
# Note DTDs are a function of DELTA t in usints of Gyr

The random number by the DTD_Ia_Power law come from the code block below to make sure that it integrates to one. 

####### Change cell type to code if want to look at normalizations
import scipy.integrate as integrate

result_power = integrate.quad(lambda x: DTD_Ia_Power(x), 0, 13.9684576774)

print 1e-3/result[0]

The cell below deals with using reshift, delta t (delay time), and lookback time (t)

In [ ]:
from scipy.interpolate import interp1d

z_inter = np.arange(0,8,0.0001)
t_inter = t_L(z_inter)

f = interp1d(t_inter, z_inter)

In [ ]:
delta_t = np.arange(0.0001,12.75,0.001)
t = delta_t[::-1]
z = f(t)

con_Ia_Power_B13 = np.convolve(cosmic_SFR(z), DTD_Ia_Power(np.array(delta_t), beta = -1.47))[0:len(t)]

con_cc_binary= np.convolve(cosmic_SFR(z), DTD_cc_binary(delta_t))[0:len(t)] 

In [ ]:
# this sets the relative fractions of Ia to CC at 30% to 70% at z = 0.001
scale_Ia = 0.25/con_Ia_Power_B13[-15]
scale_II = 0.75/con_cc_binary[-15]

# Algebra done in Kara's notebook. 
relative_Ia = 1.0/(1.+(con_cc_binary*scale_II)/(con_Ia_Power_B13*scale_Ia))
relative_II = 0.70*(1.0 - 1.0/(1.+(con_cc_binary*scale_II)/(con_Ia_Power_B13*scale_Ia)))
relative_Ibc = 0.3*(1.0 - 1.0/(1.+(con_cc_binary*scale_II)/(con_Ia_Power_B13*scale_Ia)))

In [ ]:
plt.plot(z, relative_Ia ,  label='SN Ia')
plt.plot(z,  relative_II,  label='SN II')
plt.plot(z, relative_Ibc ,  label='SN Ibc')

plt.xlabel("redshift")
plt.ylabel("Relative SN Rate")
plt.legend(loc = 2, frameon= False)
plt.xlim(0,1.5)

In [ ]:
wh_low_z = np.where(z < 1.2)

rates = np.stack((z[wh_low_z], relative_Ia[wh_low_z],relative_II[wh_low_z],relative_Ibc[wh_low_z]), axis = 1)

#np.savetxt('relative_rates.txt', rates)